In [1]:
import lightcurves.lc_utils as lu
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#%matplotlib inline

ImportError: No module named lightcurves.lc_utils

In [20]:
# Ubicacion de las curvas
# 0-1           Be_lc
# 255-256       CEPH
# 457-458       EB
# 967-968       longperiod_lc
# 1697-1698     microlensing_lc
# 2862-2863     non_variables
# 12527-12528   quasar_lc
# 12645-12646   RRL

paths = lu.get_lightcurve_paths()
#a, b = 967, 968
#a, b = 255, 256
#a, b = 2864, 2865
#a, b = 4, 5
#a,b = 2778, 2779
a,b= 12527,12528

azul = lu.open_lightcurve(paths[a])
roja = lu.open_lightcurve(paths[b])
curva = pd.concat([azul, roja], axis=1, keys=['azul', 'roja'], join='inner')

#plt.plot( azul.index, azul['mag'], '.', color = 'b')
plt.errorbar(azul.index, azul['mag'], yerr=azul['err'], fmt=".b", ecolor='r', capsize=0)
plt.show()

In [21]:
print azul.index[-1] - azul.index[0]
print 4 / 2663.0

2704.784063
0.00150206533984


In [22]:
#t_obs, y_obs, err_obs, min_time, max_time = lu.prepare_lightcurve(azul, 400)
t_obs, y_obs, err_obs, min_time, max_time = lu.prepare_lightcurve(azul, 500)

t_obs = np.ravel(t_obs)
y_obs = np.ravel(y_obs)
err_obs = np.ravel(err_obs) 

# Reduzco a la fuerza el ruido a ver si el ajuste mejora
# err_obs = err_obs/10

In [23]:
print y_obs.shape
print t_obs.shape

(500,)
(500,)


In [24]:
import george
from george import kernels

#Elijo el kernel que quiero ocupar
#kernel = 0.7**2 * kernels.ExpSquaredKernel(0.006**2)
kernel = np.var(y_obs)**2 * kernels.ExpSquaredKernel(0.006**2)

#gp = george.GP(kernel, mean=np.mean(y_obs))
gp = george.GP(kernel)

p, results = gp.optimize(t_obs, y_obs, yerr=err_obs, dims=[1])
#p, results = gp.optimize(t_obs, y_obs, dims=[1])
print p
print gp.lnlikelihood(y_obs)

Optimization terminated successfully.
[-8.36467709]
-549.714504943


Valores arbitrarios para usar en el kernel

In [25]:
mean_err = np.mean(err_obs)
var = np.var(y_obs)
#l = -(np.max(t_obs) - np.min(t_obs)) / 40
total_days = azul.index[-1] - azul.index[0]
l = 6 * (max_time - min_time) / float(total_days)   # son 6 dias segun lo observado en otros ajustes
kernel = var**2 * kernels.ExpSquaredKernel(l**2)

print mean_err, l, var

0.826383596563 0.00901586402258 1.01952217747


In [26]:
#kernel = var * kernels.ExpSquaredKernel(l**2)
gp = george.GP(kernel)
gp.compute(t_obs, yerr=err_obs)
print gp.lnlikelihood(y_obs)

-568.798378222


In [27]:
x = np.linspace(np.min(t_obs), np.max(t_obs), 500)

s = gp.sample_conditional(y_obs , t_obs)
mu, cov = gp.predict(s, x)
std = np.sqrt(np.diag(cov))

plt.plot(x, mu, color="#4682b4", alpha=0.3)
plt.errorbar(t_obs, y_obs, yerr=err_obs, fmt=".b", ecolor='r', capsize=0)

# Agrego el intervalo de confianza
plt.fill(np.concatenate([x, x[::-1]]), \
        np.concatenate([mu - 1.9600 * std,
                       (mu + 1.9600 * std)[::-1]]), \
        alpha=.5, fc='#C0C0C0', ec='None', label='95% confidence interval')
plt.show()

In [89]:
x = np.linspace(np.min(t_obs), np.max(t_obs), 500)
mu, cov = gp.predict(y_obs, x)
std = np.sqrt(np.diag(cov))

plt.plot(x, mu, color="#4682b4", alpha=0.3)
plt.errorbar(t_obs, y_obs, yerr=err_obs, fmt=".b", ecolor='r', capsize=0)

# Agrego el intervalo de confianza
plt.fill(np.concatenate([x, x[::-1]]), \
        np.concatenate([mu - 1.9600 * std,
                       (mu + 1.9600 * std)[::-1]]), \
        alpha=.5, fc='#C0C0C0', ec='None', label='95% confidence interval')
plt.show()

0.0043054715240097748